<a href="https://colab.research.google.com/github/leonardodasomci/-/blob/main/%EB%B3%BC%EB%A6%B0%EC%A0%80%EB%B0%B4%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas vectorbt matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.7 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import pandas as pd
import vectorbt as vbt

# 1. 데이터 다운로드
TICKER = 'AAPL' # 애플 주식 (원하는 종목으로 변경 가능)
START_DATE = '2020-10-01'
END_DATE = '2025-10-01'

# yfinance를 사용하여 데이터 다운로드
data = yf.download(TICKER, start=START_DATE, end=END_DATE)
price = data['Close'].squeeze() # 종가 데이터를 Series로 사용

# 2. 볼린저 밴드(BB) 지표 계산 (20일 SMA, 2.0 표준편차)
# vbt.BB를 사용하여 간단하게 BB 지표를 계산
# Create an indicator factory for Bollinger Bands
bb_indicator = vbt.IndicatorFactory.from_talib("BBANDS")

# Run the indicator on the price data
bb = bb_indicator.run(
    price,
    timeperiod=20,  # 이동평균 기간 (n=20)
    nbdevdn=2,      # 하한선 표준편차 배수 (k=2.0)
    nbdevup=2,      # 상한선 표준편차 배수 (k=2.0)
    matype=0        # 이동평균 타입 (0 for SMA)
)


# 3. 핵심 전략 로직 구현: 평균 회귀 (Mean Reversion)
# 매수 (Entry) 조건: 종가가 하한선(Lower Band)보다 낮을 때
# 매도 (Exit) 조건: 종가가 상한선(Upper Band)보다 높을 때

# 매수 시그널: 종가 < bb.lowerband
entries = price < bb.lowerband
# 매도 시그널: 종가 > bb.upperband
exits = price > bb.upperband

# 4. 백테스팅 실행 (시뮬레이션)
# initial_cash: 초기 투자금, freq: 데이터 빈도, fees: 거래 수수료 (0.1% 설정)
portfolio = vbt.Portfolio.from_signals(
    price,
    entries,
    exits,
    init_cash=10000,
    freq='D',
    fees=0.001 # 0.1% 수수료 적용
)

# 5. 결과 분석 및 출력
print(f"================ {TICKER} 볼린저 밴드 백테스팅 결과 ({START_DATE} ~ {END_DATE}) ================")

# 누적 수익률 (Cumulative Return)
total_return_formatted = f"{portfolio.total_return() * 100:.2f}%"
print(f"1. 누적 수익률: {total_return_formatted}")

# 비교 대상: 장기 보유 (Buy and Hold) 수익률
buy_and_hold_return = ((price.iloc[-1] / price.iloc[0]) - 1) * 100
print(f"   (장기 보유 수익률: {buy_and_hold_return:.2f}%)")

# 최대 낙폭 (MDD, 위험도 측정)
max_drawdown_formatted = f"{portfolio.max_drawdown() * 100:.2f}%"
print(f"2. 최대 손실 낙폭 (MDD): {max_drawdown_formatted}")

# 거래 통계 (승률, 손익비 등)
win_rate_value = portfolio.trades.win_rate()
profit_factor_value = portfolio.trades.profit_factor()

win_rate_formatted = f"{win_rate_value:.2f}%"
profit_factor_formatted = f"{profit_factor_value:.2f}"


print(f"3. 총 거래 횟수: {portfolio.trades.count()}")
print(f"4. 승률 (Win Rate): {win_rate_formatted}")
print(f"5. 손익비 (Profit Factor): {profit_factor_formatted}")

# 백테스팅 결과를 시각화 (그래프)
portfolio.plot().show()

# ----------------- 추가 분석: 거래 수수료가 없을 경우 (이상적인 상황) -----------------
portfolio_no_fees = vbt.Portfolio.from_signals(
    price,
    entries,
    exits,
    init_cash=10000,
    freq='D',
    fees=0.0 # 수수료 0%
)
no_fees_total_return_formatted = f"{portfolio_no_fees.total_return() * 100:.2f}%"
print("\n[비교] 수수료 0% 가정 시 누적 수익률: "
      f"{no_fees_total_return_formatted}")

/tmp/ipython-input-1243501087.py:11: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


================ AAPL 볼린저 밴드 백테스팅 결과 (2020-10-01 ~ 2025-10-01) ================
1. 누적 수익률: 69.46%
   (장기 보유 수익률: 124.14%)
2. 최대 손실 낙폭 (MDD): -24.28%
3. 총 거래 횟수: 16
4. 승률 (Win Rate): 0.75%
5. 손익비 (Profit Factor): 3.97



[비교] 수수료 0% 가정 시 누적 수익률: 74.97%


In [ ]:
!pip install -q ta-lib